# Задание 2.1 - Нейронные сети

В этом задании вы реализуете и натренируете настоящую нейроную сеть своими руками!

В некотором смысле это будет расширением прошлого задания - нам нужно просто составить несколько линейных классификаторов вместе!

<img src="https://i.redd.it/n9fgba8b0qr01.png" alt="Stack_more_layers" width="400px"/>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
from dataset import load_svhn, random_split_train_val
from gradient_check import check_layer_gradient, check_layer_param_gradient, check_model_gradient
from layers import FullyConnectedLayer, ReLULayer
from model import TwoLayerNet
from trainer import Trainer, Dataset
from optim import SGD, MomentumSGD
from metrics import multiclass_accuracy

# Загружаем данные

И разделяем их на training и validation.

In [3]:
def prepare_for_neural_network(train_X, test_X):
    train_flat = train_X.reshape(train_X.shape[0], -1).astype(np.float) / 255.0
    test_flat = test_X.reshape(test_X.shape[0], -1).astype(np.float) / 255.0
    
    # Subtract mean
    mean_image = np.mean(train_flat, axis = 0)
    train_flat -= mean_image
    test_flat -= mean_image
    
    return train_flat, test_flat
    
train_X, train_y, test_X, test_y = load_svhn("../assignment1/data", max_train=10000, max_test=1000)    
train_X, test_X = prepare_for_neural_network(train_X, test_X)
# Split train into train and val
train_X, train_y, val_X, val_y = random_split_train_val(train_X, train_y, num_val = 1000)

<ipython-input-3-3dff2068dd0c>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_flat = train_X.reshape(train_X.shape[0], -1).astype(np.float) / 255.0
<ipython-input-3-3dff2068dd0c>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_flat = test_X.reshape(test_X.shape[0], -1).astype(np.float) / 255.0


# Как всегда, начинаем с кирпичиков

Мы будем реализовывать необходимые нам слои по очереди. Каждый слой должен реализовать:
- прямой проход (forward pass), который генерирует выход слоя по входу и запоминает необходимые данные
- обратный проход (backward pass), который получает градиент по выходу слоя и вычисляет градиент по входу и по параметрам

Начнем с ReLU, у которого параметров нет.

In [4]:
# TODO: Implement ReLULayer layer in layers.py
# Note: you'll need to copy implementation of the gradient_check function from the previous assignment

X = np.array([[1,-2,3],
              [-1, 2, 0.1]
              ])

assert check_layer_gradient(ReLULayer(), X)

Gradient check passed!


А теперь реализуем полносвязный слой (fully connected layer), у которого будет два массива параметров: W (weights) и B (bias).

Все параметры наши слои будут использовать для параметров специальный класс `Param`, в котором будут храниться значения параметров и градиенты этих параметров, вычисляемые во время обратного прохода.

Это даст возможность аккумулировать (суммировать) градиенты из разных частей функции потерь, например, из cross-entropy loss и regularization loss.

In [85]:
# TODO: Implement FullyConnected layer forward and backward methods
assert check_layer_gradient(FullyConnectedLayer(3, 4), X)
# TODO: Implement storing gradients for W and B
assert check_layer_param_gradient(FullyConnectedLayer(3, 4), X, 'W')
assert check_layer_param_gradient(FullyConnectedLayer(3, 4), X, 'B')

Gradient check passed!
Gradient check passed!
Gradient check passed!


## Создаем нейронную сеть

Теперь мы реализуем простейшую нейронную сеть с двумя полносвязным слоями и нелинейностью ReLU. Реализуйте функцию `compute_loss_and_gradients`, она должна запустить прямой и обратный проход через оба слоя для вычисления градиентов.

Не забудьте реализовать очистку градиентов в начале функции.

In [27]:
# TODO: In model.py, implement compute_loss_and_gradients function
model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 3, reg = 0)
loss = model.compute_loss_and_gradients(train_X[:2], train_y[:2])

# TODO Now implement backward pass and aggregate all of the params
check_model_gradient(model, train_X[:2], train_y[:2])

Checking gradient for linear_1_W
Gradient check passed!
Checking gradient for linear_1_B
Gradient check passed!
Checking gradient for linear_2_W
Gradient check passed!
Checking gradient for linear_2_B
Gradient check passed!


True

Теперь добавьте к модели регуляризацию - она должна прибавляться к loss и делать свой вклад в градиенты.

In [22]:
# TODO Now implement l2 regularization in the forward and backward pass
model_with_reg = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 3, reg = 1e1)
loss_with_reg = model_with_reg.compute_loss_and_gradients(train_X[5:10], train_y[5:10])
assert loss_with_reg > loss and not np.isclose(loss_with_reg, loss), \
    "Loss with regularization (%2.4f) should be higher than without it (%2.4f)!" % (loss, loss_with_reg)

check_model_gradient(model_with_reg, train_X[5:10], train_y[5:10])

Checking gradient for linear_1_W
Gradient check passed!
Checking gradient for linear_1_B
Gradient check passed!
Checking gradient for linear_2_W
Gradient check passed!
Checking gradient for linear_2_B
Gradient check passed!


True

Также реализуем функцию предсказания (вычисления значения) модели на новых данных.

Какое значение точности мы ожидаем увидеть до начала тренировки?

In [8]:
# Finally, implement predict function!

# TODO: Implement predict function
# What would be the value we expect?
multiclass_accuracy(model_with_reg.predict(train_X[:30]), train_y[:30]) 

0.1

# Допишем код для процесса тренировки

Если все реализовано корректно, значение функции ошибки должно уменьшаться с каждой эпохой, пусть и медленно. Не беспокойтесь пока про validation accuracy.

In [113]:
model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e1)
dataset = Dataset(train_X, train_y, val_X, val_y)
trainer = Trainer(model, dataset, SGD(), learning_rate = 1e-2)

# TODO Implement missing pieces in Trainer.fit function
# You should expect loss to go down every epoch, even if it's slow
loss_history, train_history, val_history = trainer.fit()

KeyboardInterrupt: 

In [77]:
from model import OneLayerNet

model = OneLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e1)
dataset = Dataset(train_X, train_y, val_X, val_y)
trainer = Trainer(model, dataset, SGD(), learning_rate = 1e-2)

# TODO Implement missing pieces in Trainer.fit function
# You should expect loss to go down every epoch, even if it's slow
loss_history, train_history, val_history = trainer.fit()

Loss: 2.304045, Train accuracy: 0.138778, val accuracy: 0.133000
Loss: 2.305318, Train accuracy: 0.170222, val accuracy: 0.174000
Loss: 2.304309, Train accuracy: 0.131444, val accuracy: 0.131000
Loss: 2.333627, Train accuracy: 0.099444, val accuracy: 0.101000
Loss: 2.316517, Train accuracy: 0.118556, val accuracy: 0.115000
Loss: 2.275625, Train accuracy: 0.157000, val accuracy: 0.172000
Loss: 2.323468, Train accuracy: 0.154556, val accuracy: 0.161000
Loss: 2.300946, Train accuracy: 0.159444, val accuracy: 0.154000
Loss: 2.335730, Train accuracy: 0.127333, val accuracy: 0.112000
Loss: 2.318063, Train accuracy: 0.123556, val accuracy: 0.127000
Loss: 2.324191, Train accuracy: 0.138111, val accuracy: 0.147000
Loss: 2.301464, Train accuracy: 0.093667, val accuracy: 0.098000
Loss: 2.292479, Train accuracy: 0.125667, val accuracy: 0.140000
Loss: 2.368481, Train accuracy: 0.096111, val accuracy: 0.095000
Loss: 2.338286, Train accuracy: 0.134889, val accuracy: 0.161000
Loss: 2.289386, Train acc

In [97]:
from layers import ReLULayer
r = ReLULayer()
x = np.array([[1,2,3],[0.1,-2,-3],[-3,2,-5]], dtype=float)
check_layer_gradient(r, x)
r.dX
r.forward(np.array([1,2,-2], dtype=float))

Gradient check passed!


array([1., 2., 0.])

In [114]:
model = OneLayerNet(n_input = 2, n_output = 4, hidden_layer_size = 10, reg = 0)
dataset = Dataset(np.array([[0,0], [0,1], [1,0], [1,1]]),
                    np.array([0,1,2,3]), 
                    np.array([[0,0], [0,1], [1,0], [1,1]]),
                    np.array([0,1,2,3])
                    )
trainer = Trainer(model, dataset, SGD(), learning_rate = 1e-4, batch_size=1, num_epochs=200)

# TODO Implement missing pieces in Trainer.fit function
# You should expect loss to go down every epoch, even if it's slow
loss_history, train_history, val_history = trainer.fit()
model.params()

Loss: 1.386644, Train accuracy: 0.000000, val accuracy: 0.000000
Loss: 1.386644, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.386644, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.385479, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.386644, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.384027, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.386644, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.385280, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.386643, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.385180, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.383529, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.383430, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.386643, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.383231, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.384931, Train accuracy: 0.250000, val accuracy: 0.250000
Loss: 1.386643, Train acc

{'linear_1_W': <layers.Param at 0x1bcbb8779a0>,
 'linear_1_B': <layers.Param at 0x1bcbb8776d0>}

In [121]:
model = TwoLayerNet(n_input = 2, n_output = 4, hidden_layer_size = 10, reg = 0)
dataset = Dataset(np.array([[0,0], [0,1], [1,0], [1,1]]),
                    np.array([0,1,2,3]), 
                    np.array([[0,0], [0,1], [1,0], [1,1]]),
                    np.array([0,1,2,3])
                    )
trainer = Trainer(model, dataset, SGD(), learning_rate = 1e-4, batch_size=1, num_epochs=1)

# TODO Implement missing pieces in Trainer.fit function
# You should expect loss to go down every epoch, even if it's slow
loss_history, train_history, val_history = trainer.fit()
# print(model.params()['linear_1_B'].value)
# p = model.params()
# p['linear_1_B'].value = np.zeros_like(model.params()['linear_1_B'].value)
# print(model.params()['linear_1_B'].value)

Loss: 1.386831, Train accuracy: 0.250000, val accuracy: 0.250000
[[-3.71569086e-05  5.02231275e-04 -1.29463346e-03 -8.94378595e-04
   1.26249876e-03  3.44281436e-05 -1.67781688e-03 -2.90937595e-04
   1.74909653e-04  4.70344631e-04]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [73]:
model.predict(np.array([[0,0], [0,1], [1,0], [1,1]]))

array([0, 0, 0, 0], dtype=int64)

In [48]:
for name, par in model.params().items():
    print(f"{name}: {par.value}")

linear_1_W: [[ 3.09012500e-08 -1.77842373e-07]
 [ 5.55417550e-08 -7.10245543e-08]]
linear_1_B: [[3.87971842e-08 9.50328659e-08]]
linear_2_W: [[-2.21426754e-07 -2.00618510e-08 -8.36578081e-09 -3.11341442e-08]
 [-1.09457498e-07  6.61053296e-08 -2.13037927e-08 -1.34646267e-07]]
linear_2_B: [[ 0.00196292 -0.00022334 -0.00031654 -0.00142303]]


In [ ]:
plt.plot(train_history)
plt.plot(val_history)

# Улучшаем процесс тренировки

Мы реализуем несколько ключевых оптимизаций, необходимых для тренировки современных нейросетей.

## Уменьшение скорости обучения (learning rate decay)

Одна из необходимых оптимизаций во время тренировки нейронных сетей - постепенное уменьшение скорости обучения по мере тренировки.

Один из стандартных методов - уменьшение скорости обучения (learning rate) каждые N эпох на коэффициент d (часто называемый decay). Значения N и d, как всегда, являются гиперпараметрами и должны подбираться на основе эффективности на проверочных данных (validation data). 

В нашем случае N будет равным 1.

In [24]:
# TODO Implement learning rate decay inside Trainer.fit method
# Decay should happen once per epoch

model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e-1)
dataset = Dataset(train_X, train_y, val_X, val_y)
trainer = Trainer(model, dataset, SGD(), learning_rate_decay=0.99)

initial_learning_rate = trainer.learning_rate
loss_history, train_history, val_history = trainer.fit()

assert trainer.learning_rate < initial_learning_rate, "Learning rate should've been reduced"
assert trainer.learning_rate > 0.5*initial_learning_rate, "Learning rate shouldn'tve been reduced that much!"

Loss: 2.303905, Train accuracy: 0.196667, val accuracy: 0.206000


KeyboardInterrupt: 

# Накопление импульса (Momentum SGD)

Другой большой класс оптимизаций - использование более эффективных методов градиентного спуска. Мы реализуем один из них - накопление импульса (Momentum SGD).

Этот метод хранит скорость движения, использует градиент для ее изменения на каждом шаге, и изменяет веса пропорционально значению скорости.
(Физическая аналогия: Вместо скорости градиенты теперь будут задавать ускорение, но будет присутствовать сила трения.)

```
velocity = momentum * velocity - learning_rate * gradient 
w = w + velocity
```

`momentum` здесь коэффициент затухания, который тоже является гиперпараметром (к счастью, для него часто есть хорошее значение по умолчанию, типичный диапазон -- 0.8-0.99).

Несколько полезных ссылок, где метод разбирается более подробно:  
http://cs231n.github.io/neural-networks-3/#sgd  
https://distill.pub/2017/momentum/

In [ ]:
# TODO: Implement MomentumSGD.update function in optim.py

model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e-1)
dataset = Dataset(train_X, train_y, val_X, val_y)
trainer = Trainer(model, dataset, MomentumSGD(), learning_rate=1e-4, learning_rate_decay=0.99)

# You should see even better results than before!
loss_history, train_history, val_history = trainer.fit()

# Ну что, давайте уже тренировать сеть!

## Последний тест - переобучимся (overfit) на маленьком наборе данных

Хороший способ проверить, все ли реализовано корректно - переобучить сеть на маленьком наборе данных.  
Наша модель обладает достаточной мощностью, чтобы приблизить маленький набор данных идеально, поэтому мы ожидаем, что на нем мы быстро дойдем до 100% точности на тренировочном наборе. 

Если этого не происходит, то где-то была допущена ошибка!

In [ ]:
data_size = 15
model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e-1)
dataset = Dataset(train_X[:data_size], train_y[:data_size], val_X[:data_size], val_y[:data_size])
trainer = Trainer(model, dataset, SGD(), learning_rate=1e-1, num_epochs=150, batch_size=5)

# You should expect this to reach 1.0 training accuracy 
loss_history, train_history, val_history = trainer.fit()

Теперь найдем гипепараметры, для которых этот процесс сходится быстрее.
Если все реализовано корректно, то существуют параметры, при которых процесс сходится в **20** эпох или еще быстрее.
Найдите их!

In [ ]:
# Now, tweak some hyper parameters and make it train to 1.0 accuracy in 20 epochs or less

model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e-1)
dataset = Dataset(train_X[:data_size], train_y[:data_size], val_X[:data_size], val_y[:data_size])
# TODO: Change any hyperparamers or optimizators to reach training accuracy in 20 epochs
trainer = Trainer(model, dataset, SGD(), learning_rate=1e-1, num_epochs=20, batch_size=5)

loss_history, train_history, val_history = trainer.fit()

# Итак, основное мероприятие!

Натренируйте лучшую нейросеть! Можно добавлять и изменять параметры, менять количество нейронов в слоях сети и как угодно экспериментировать. 

Добейтесь точности лучше **60%** на validation set.

In [ ]:
# Let's train the best one-hidden-layer network we can

learning_rates = 1e-4
reg_strength = 1e-3
learning_rate_decay = 0.999
hidden_layer_size = 128
num_epochs = 200
batch_size = 64

best_classifier = None
best_val_accuracy = None

loss_history = []
train_history = []
val_history = []

# TODO find the best hyperparameters to train the network
# Don't hesitate to add new values to the arrays above, perform experiments, use any tricks you want
# You should expect to get to at least 40% of valudation accuracy
# Save loss/train/history of the best classifier to the variables above

print('best validation accuracy achieved: %f' % best_val_accuracy)

In [ ]:
plt.figure(figsize=(15, 7))
plt.subplot(211)
plt.title("Loss")
plt.plot(loss_history)
plt.subplot(212)
plt.title("Train/validation accuracy")
plt.plot(train_history)
plt.plot(val_history)

# Как обычно, посмотрим, как наша лучшая модель работает на тестовых данных

In [ ]:
test_pred = best_classifier.predict(test_X)
test_accuracy = multiclass_accuracy(test_pred, test_y)
print('Neural net test set accuracy: %f' % (test_accuracy, ))